In [1]:
#Data source
dpath = "/home/ubuntu/data/"
import pandas as pd, numpy as np

#Preprocessing steps
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

#Metrics to use
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score
beta = 2 #favors recall 
from sklearn.model_selection import cross_val_score

#Classifiers
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [2]:
from __future__ import print_function

In [3]:
#List of all classifiers that will be trained
classifiers = [LogisticRegression(n_jobs=-1), RandomForestClassifier(n_jobs=-1), GaussianNB(), 
               MLPClassifier(), KNeighborsClassifier(n_jobs=-1), SVC(probability=True)]

In [4]:
#List of all features that will be combined in late fusion
feat_to_combine = ["AutoColorCorrelogram", "BinaryPatternsPyramid", "EdgeHistogram", "FCTH", "FuzzyColorHistogram", 
                       "FuzzyOpponentHistogram", "Gabor", "JCD", "JpegCoefficientHistogram", "LocalBinaryPatterns", 
                       "LuminanceLayout", "OpponentHistogram", "PHOG", "RotationInvariantLocalBinaryPatterns", "ScalableColor", 
                       "SimpleColorHistogram", "Tamura", "CEDD", "ColorLayout"]

In [5]:
#Loading of the train and test set descriptors
train_set = pd.read_csv(dpath + "train_set.csv")
test_set = pd.read_csv(dpath + "test_set.csv")

In [6]:
#Creation of the feature set list

#We'll load all the feature files, preprocess the data and store it as tuples of train,test sets

feature_sets = {}
for feat in feat_to_combine:
    features_train = pd.read_csv(dpath + "features_%s_%s.csv" % (feat, "train"), header=None, skiprows=1)
    features_train = features_train.drop(features_train.columns[0], axis=1)
    features_test = pd.read_csv(dpath + "features_%s_%s.csv" % (feat, "test"), header=None, skiprows=1) 
    features_test = features_test.drop(features_test.columns[0], axis=1)
    
    #Feature transformation pipeline: MinMaxScaling + PCA (with the number of components necesary to explain 99% of variance)
    processing_pipeline = make_pipeline( MinMaxScaler(), PCA( n_components = 0.99 ) )
    X_features_train = processing_pipeline.fit_transform(features_train.as_matrix())
    X_features_test = processing_pipeline.transform(features_test.as_matrix())
    
    feature_sets[feat] = (X_features_train, X_features_test)


In [7]:
#Function to obtain the classifier name from their string representation
def classifier_name(classifier):
    return str(classifier).split("(")[0]

In [8]:
#ids and targets

id_train = train_set["Image"]
y_train = train_set["Class"]

id_test = test_set["Image"]
y_test = test_set["Class"]

In [9]:
def train_and_predict(classifier, X_features_train, y_train, X_features_test):
    
    #Obtaining cross-validated metrics
    score = cross_val_score(classifier, X_features_train, y_train, scoring="accuracy", cv=4, n_jobs=4).mean()

    #Training with the full train set and making predictions on the test set
    classifier = classifier.fit(X_features_train, y_train)
    prediction = classifier.predict_proba(X_features_test)
    
    return prediction, score

In [10]:
%%time

metrics = pd.DataFrame(columns=["model", "accuracy", "F1" ,"precision", "recall", "Fbeta"])
model_scores = {}
model_predictions = {}

for classifier in classifiers: 
    
    model_name = classifier_name(classifier)
    print("--- Model:", model_name)
    
    l1_scores = {}
    l1_test_predictions = {}
    
    for feat_set in feat_to_combine:
        print("- features:", feat_set)
        features_train, features_test = feature_sets[feat_set]
        
        prediction, score = train_and_predict(classifier, features_train, y_train, features_test)
        
        l1_scores[feat_set] = score
        l1_test_predictions[feat_set] = prediction
        
    
    model_scores[model_name] = l1_scores
    l1_test_predictions = pd.Panel(l1_test_predictions)
    
    #To combine predictions, we can average the probability predictions and choose the highest
    prob_ave = l1_test_predictions.mean(axis=0).as_matrix()
    pred_classes_ave = [classifier.classes_[i] for i in np.argmax(prob_ave, axis=1)]
    
    m_ave = {"model": model_name+"_Average", 
         "accuracy": accuracy_score(y_test, pred_classes_ave), 
         "F1": f1_score(y_test, pred_classes_ave, average='macro'), 
         "precision": precision_score(y_test, pred_classes_ave, average='macro'), 
         "recall": recall_score(y_test, pred_classes_ave, average='macro'),
         "Fbeta": fbeta_score(y_test, pred_classes_ave, beta=beta, average='macro'), 
           }
    metrics = metrics.append(m_ave, ignore_index=True)
    model_predictions[model_name+"_Average"] = pred_classes_ave
    
    
    #We can also do a weighted average, by each individual accuracy
    for k in l1_scores.keys():
        s = l1_scores[k]
        l1_test_predictions[k] = l1_test_predictions[k] * s
    
    prob_weight = l1_test_predictions.sum(axis=0).as_matrix()
    pred_classes_weight = [classifier.classes_[i] for i in np.argmax(prob_weight, axis=1)]
    
    m_weight = {"model": model_name+"_Weighted", 
         "accuracy": accuracy_score(y_test, pred_classes_weight), 
         "F1": f1_score(y_test, pred_classes_weight, average='macro'), 
         "precision": precision_score(y_test, pred_classes_weight, average='macro'), 
         "recall": recall_score(y_test, pred_classes_weight, average='macro'),
         "Fbeta": fbeta_score(y_test, pred_classes_weight, beta=beta, average='macro'), 
           }
    metrics = metrics.append(m_weight, ignore_index=True)
    model_predictions[model_name+"_Weighted"] = pred_classes_weight
    


--- Model: LogisticRegression
- features: AutoColorCorrelogram
- features: BinaryPatternsPyramid
- features: EdgeHistogram
- features: FCTH
- features: FuzzyColorHistogram
- features: FuzzyOpponentHistogram
- features: Gabor
- features: JCD
- features: JpegCoefficientHistogram
- features: LocalBinaryPatterns
- features: LuminanceLayout
- features: OpponentHistogram
- features: PHOG
- features: RotationInvariantLocalBinaryPatterns
- features: ScalableColor
- features: SimpleColorHistogram
- features: Tamura
- features: CEDD
- features: ColorLayout
--- Model: RandomForestClassifier
- features: AutoColorCorrelogram
- features: BinaryPatternsPyramid
- features: EdgeHistogram
- features: FCTH
- features: FuzzyColorHistogram
- features: FuzzyOpponentHistogram
- features: Gabor
- features: JCD
- features: JpegCoefficientHistogram
- features: LocalBinaryPatterns
- features: LuminanceLayout
- features: OpponentHistogram
- features: PHOG
- features: RotationInvariantLocalBinaryPatterns
- feature

/usr/local/lib/python2.7/dist-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


- features: BinaryPatternsPyramid


/usr/local/lib/python2.7/dist-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


- features: EdgeHistogram
- features: FCTH
- features: FuzzyColorHistogram
- features: FuzzyOpponentHistogram
- features: Gabor
- features: JCD
- features: JpegCoefficientHistogram
- features: LocalBinaryPatterns
- features: LuminanceLayout
- features: OpponentHistogram
- features: PHOG
- features: RotationInvariantLocalBinaryPatterns
- features: ScalableColor
- features: SimpleColorHistogram
- features: Tamura
- features: CEDD
- features: ColorLayout
--- Model: KNeighborsClassifier
- features: AutoColorCorrelogram
- features: BinaryPatternsPyramid
- features: EdgeHistogram
- features: FCTH
- features: FuzzyColorHistogram
- features: FuzzyOpponentHistogram
- features: Gabor
- features: JCD
- features: JpegCoefficientHistogram
- features: LocalBinaryPatterns
- features: LuminanceLayout
- features: OpponentHistogram
- features: PHOG
- features: RotationInvariantLocalBinaryPatterns
- features: ScalableColor
- features: SimpleColorHistogram
- features: Tamura
- features: CEDD
- features: C

In [11]:
metrics

model  accuracy        F1  precision  recall  \
0        LogisticRegression_Average    0.8815  0.880465   0.885320  0.8815   
1       LogisticRegression_Weighted    0.8820  0.881091   0.885241  0.8820   
2    RandomForestClassifier_Average    0.8250  0.823025   0.827895  0.8250   
3   RandomForestClassifier_Weighted    0.8305  0.828497   0.834012  0.8305   
4                GaussianNB_Average    0.8300  0.829121   0.833758  0.8300   
5               GaussianNB_Weighted    0.8365  0.836285   0.841593  0.8365   
6             MLPClassifier_Average    0.8930  0.892540   0.894517  0.8930   
7            MLPClassifier_Weighted    0.8935  0.892984   0.895004  0.8935   
8      KNeighborsClassifier_Average    0.8355  0.833223   0.841968  0.8355   
9     KNeighborsClassifier_Weighted    0.8350  0.832902   0.840948  0.8350   
10                      SVC_Average    0.8590  0.858065   0.861984  0.8590   
11                     SVC_Weighted    0.8580  0.857184   0.860582  0.8580   

       Fbeta  
0   0.880410  
1   0.881053  
2   0.823460  
3   0.828882  
4   0.829045  
5   0.835799  
6   0.892534  
7   0.892999  
8   0.833423  
9   0.833066  
10  0.858091  
11  0.857200

In [12]:
metrics.sort_values(by=["accuracy", "F1"], ascending=False).to_csv(dpath+"metrics_late_fusion_Set2-PCA.csv", index=False)

In [13]:
model_scores = pd.DataFrame(model_scores)
model_scores

GaussianNB  KNeighborsClassifier  \
AutoColorCorrelogram                    0.638497              0.650050   
BinaryPatternsPyramid                   0.263489              0.452413   
CEDD                                    0.618480              0.662490   
ColorLayout                             0.646969              0.616831   
EdgeHistogram                           0.494848              0.482071   
FCTH                                    0.578069              0.616119   
FuzzyColorHistogram                     0.635905              0.612503   
FuzzyOpponentHistogram                  0.457317              0.579477   
Gabor                                   0.395041              0.399402   
JCD                                     0.643465              0.658994   
JpegCoefficientHistogram                0.538322              0.510145   
LocalBinaryPatterns                     0.284362              0.372472   
LuminanceLayout                         0.233623              0.336950   
OpponentHistogram                       0.471326              0.531986   
PHOG                                    0.312068              0.395345   
RotationInvariantLocalBinaryPatterns    0.342910              0.374000   
ScalableColor                           0.511529              0.554443   
SimpleColorHistogram                    0.521001              0.622040   
Tamura                                  0.396857              0.342950   

                                      LogisticRegression  MLPClassifier  \
AutoColorCorrelogram                            0.706893       0.717942   
BinaryPatternsPyramid                           0.518489       0.490047   
CEDD                                            0.686516       0.689580   
ColorLayout                                     0.645889       0.680324   
EdgeHistogram                                   0.538418       0.532970   
FCTH                                            0.674435       0.680500   
FuzzyColorHistogram                             0.653922       0.694404   
FuzzyOpponentHistogram                          0.548923       0.645881   
Gabor                                           0.366015       0.429491   
JCD                                             0.701045       0.697493   
JpegCoefficientHistogram                        0.588742       0.649410   
LocalBinaryPatterns                             0.397401       0.425435   
LuminanceLayout                                 0.358431       0.373440   
OpponentHistogram                               0.527898       0.586310   
PHOG                                            0.484799       0.484887   
RotationInvariantLocalBinaryPatterns            0.360927       0.420955   
ScalableColor                                   0.541363       0.571949   
SimpleColorHistogram                            0.631880       0.668387   
Tamura                                          0.390401       0.418419   

                                      RandomForestClassifier       SVC  
AutoColorCorrelogram                                0.599502  0.665971  
BinaryPatternsPyramid                               0.296891  0.464966  
CEDD                                                0.598926  0.645945  
ColorLayout                                         0.617432  0.603015  
EdgeHistogram                                       0.372904  0.530938  
FCTH                                                0.612511  0.617952  
FuzzyColorHistogram                                 0.621992  0.637993  
FuzzyOpponentHistogram                              0.587846  0.518065  
Gabor                                               0.380528  0.433436  
JCD                                                 0.601510  0.652498  
JpegCoefficientHistogram                            0.535914  0.491383  
LocalBinaryPatterns                                 0.359927  0.307972  
LuminanceLayout                                     0.316404  0.272593  
OpponentHistogram                 

In [14]:
model_scores.to_csv(dpath+"individual_metrics_late_fusion_Set2-PCA.csv")

In [15]:
#Combining predictions of all classifiers
pred_all = pd.DataFrame(model_predictions).mode(axis=1)[0]

{"model": "All Classifiers Mode", 
         "accuracy": accuracy_score(y_test, pred_all), 
         "F1": f1_score(y_test, pred_all, average='macro'), 
         "precision": precision_score(y_test, pred_all, average='macro'), 
         "recall": recall_score(y_test, pred_all, average='macro'),
         "Fbeta": fbeta_score(y_test, pred_all, beta=beta, average='macro'), 
           }

{'F1': 0.89152264121129599,
 'Fbeta': 0.89152456326600882,
 'accuracy': 0.89200000000000002,
 'model': 'All Classifiers Mode',
 'precision': 0.89348394930002994,
 'recall': 0.8919999999999999}